In [1]:
# Begin by cloning the development branch of MAST-ML to the Colab session
# !git clone --single-branch --branch dev_Ryan_2020-12-21 https://github.com/uw-cmg/MAST-ML

In [2]:
# Next, we install the required dependencies of MAST-ML to our Colab session
# !pip install -r MAST-ML/requirements.txt

In [3]:
# Sync your Google drive to Colab so that we can save MAST-ML results to our Google
# Drive. If we save to the Colab session, the data will be deleted when the session 
# ends.
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)

In [144]:
# We need to add the MAST-ML folder to our sys path so that python can find the modules
import sys
sys.path.append('MAST-ML')
import pandas as pd
import numpy as np

In [145]:
# Here we import the MAST-ML modules used in this tutorial
from mastml.mastml import Mastml
from mastml.datasets import LocalDatasets, DataCleaning,  LocalDatasets
from mastml.preprocessing import SklearnPreprocessor
from mastml.models import SklearnModel
from mastml.data_splitters import SklearnDataSplitter, NoSplit
from mastml.feature_selectors import EnsembleModelFeatureSelector, NoSelect

In [160]:
# Set the name of the savepath to save MAST-ML results to
# SAVEPATH = 'drive/MyDrive/MASTML_results_2021-03-01'
SAVEPATH = '/Users/bernie/Desktop/course_material/SUMMER21/skunk_summer/June/MASTML_Fried_6ktest10'
# Initialize the MAST-ML run, write savepath
mastml = Mastml(savepath=SAVEPATH)
savepath = mastml.get_savepath

In [161]:
savepath

'/Users/bernie/Desktop/course_material/SUMMER21/skunk_summer/June/MASTML_Fried_6ktest10'

In [162]:
import os
path = os.path.join('./data','fried_6k_test10.csv') # Supercon_data_features_selected.xlsx
path

'./data/fried_6k_test10.csv'

In [163]:
total_df = pd.read_csv(path)
features = list(total_df.columns)[:-1] 
total_df = total_df[features]
total_df

,x0,x1,x2,x3,x4,y
0,0.205116,0.707143,0.624922,0.756035,0.299970,13.773013
1,0.423566,0.283326,0.491772,0.397476,0.753117,11.423156
2,0.202296,0.358321,0.325332,0.933607,0.472139,14.564564
3,0.213097,0.765921,0.556923,0.397080,0.568637,11.784602
4,0.613091,0.602007,0.896572,0.026473,0.419895,14.675679
...,...,...,...,...,...,...
3295,0.963984,0.155984,0.983608,0.081212,0.039612,10.237858
3296,0.987380,0.962316,0.997030,0.732961,0.031129,13.985048
3297,0.134506,0.007682,0.982191,0.047674,0.105378,5.686248
3298,0.998521,0.139649,0.070542,0.073467,0.014744,8.739013


In [164]:
len( features )

6

In [165]:
# ds = LocalDatasets('friedman_data_200*3_sortx0.csv',
#                     feature_names = ['x0','x1', 'x2', 'x3', 'x4'],
#                     target = 'y',
#                     testdata_columns = ['X0'])
ds = LocalDatasets( path, # path to the csv file include train and test data
                    feature_names = features , # features we want to fit with the model
                    target = 'y',              # targeted value is y
                    testdata_columns = ['leave_out']  # the column specifies leaveout (0:train, 1: test)
                  )
df = ds.load_data()
df['X_testdata'] # it has the index for each test data in our entire dataset

array([[3000, 3001, 3002, 3003, 3004, 3005, 3006, 3007, 3008, 3009, 3010,
        3011, 3012, 3013, 3014, 3015, 3016, 3017, 3018, 3019, 3020, 3021,
        3022, 3023, 3024, 3025, 3026, 3027, 3028, 3029, 3030, 3031, 3032,
        3033, 3034, 3035, 3036, 3037, 3038, 3039, 3040, 3041, 3042, 3043,
        3044, 3045, 3046, 3047, 3048, 3049, 3050, 3051, 3052, 3053, 3054,
        3055, 3056, 3057, 3058, 3059, 3060, 3061, 3062, 3063, 3064, 3065,
        3066, 3067, 3068, 3069, 3070, 3071, 3072, 3073, 3074, 3075, 3076,
        3077, 3078, 3079, 3080, 3081, 3082, 3083, 3084, 3085, 3086, 3087,
        3088, 3089, 3090, 3091, 3092, 3093, 3094, 3095, 3096, 3097, 3098,
        3099, 3100, 3101, 3102, 3103, 3104, 3105, 3106, 3107, 3108, 3109,
        3110, 3111, 3112, 3113, 3114, 3115, 3116, 3117, 3118, 3119, 3120,
        3121, 3122, 3123, 3124, 3125, 3126, 3127, 3128, 3129, 3130, 3131,
        3132, 3133, 3134, 3135, 3136, 3137, 3138, 3139, 3140, 3141, 3142,
        3143, 3144, 3145, 3146, 3147, 

In [166]:
from sklearn.datasets import make_friedman1
import pandas as pd

def friedman_np_to_df(X,y):
    return pd.DataFrame(X, columns= features), pd.Series(y)
X = df['X']
y = df['y']
X,y = friedman_np_to_df(X,y)

leave_indicies = df['X_testdata']
# since leaveout_inds of MASTML requires format: [array([a,b,c])]
# instead of np.array([ [a,b,c] ]), we reconstruct it
X_test = [ leave_indicies[0] ]

In [167]:
X_test

[array([3000, 3001, 3002, 3003, 3004, 3005, 3006, 3007, 3008, 3009, 3010,
        3011, 3012, 3013, 3014, 3015, 3016, 3017, 3018, 3019, 3020, 3021,
        3022, 3023, 3024, 3025, 3026, 3027, 3028, 3029, 3030, 3031, 3032,
        3033, 3034, 3035, 3036, 3037, 3038, 3039, 3040, 3041, 3042, 3043,
        3044, 3045, 3046, 3047, 3048, 3049, 3050, 3051, 3052, 3053, 3054,
        3055, 3056, 3057, 3058, 3059, 3060, 3061, 3062, 3063, 3064, 3065,
        3066, 3067, 3068, 3069, 3070, 3071, 3072, 3073, 3074, 3075, 3076,
        3077, 3078, 3079, 3080, 3081, 3082, 3083, 3084, 3085, 3086, 3087,
        3088, 3089, 3090, 3091, 3092, 3093, 3094, 3095, 3096, 3097, 3098,
        3099, 3100, 3101, 3102, 3103, 3104, 3105, 3106, 3107, 3108, 3109,
        3110, 3111, 3112, 3113, 3114, 3115, 3116, 3117, 3118, 3119, 3120,
        3121, 3122, 3123, 3124, 3125, 3126, 3127, 3128, 3129, 3130, 3131,
        3132, 3133, 3134, 3135, 3136, 3137, 3138, 3139, 3140, 3141, 3142,
        3143, 3144, 3145, 3146, 3147, 

In [168]:
# below is the setup, just run it

In [169]:

# Here is where we can specify some of the core MAST-ML options, like how to 
# preprocess our data, what models to use in fitting, and how to select features.
# In addition, we would also specify how to optimize model hyperparameters, but
# we don't need to do that for this first tutorial. Lastly, we can decide which
# metrics to evaluate in our fits.

# Here, we define two models. The first is a random forest model, the second is
# a Gaussian process model. The scikit-learn model name can just be given as
# a string matching the model name in the "model" field. The remaining arguments
# are the parameters to pass to the model. If no parameters are given, default
# values are used.
# model1 = SklearnModel(model='RandomForestRegressor', n_estimators=150, max_depth=30)
model1 = SklearnModel(model='RandomForestRegressor', n_estimators=500, max_depth=30)
model2 = SklearnModel(model='GaussianProcessRegressor', kernel='ConstantKernel*RBF', n_restarts_optimizer=10)
# MAST-ML takes a list of the models as input.
models = [model1, model2]
# models = [model1]
# Here, we define our preprocessing function. We are just going to use the basic
# StandardScaler in scikit-learn to normalize each column to have mean zero and
# standard deviation of one.
preprocessor = SklearnPreprocessor(preprocessor='StandardScaler', as_frame=True)

# Next, we define a couple ways to select features. The first method, NoSelect,
# just uses all features in the X feature set. The second method, EnsembleModelFeatureSelector,
# uses a random forest model to select the top (here, 20) features based on 
# ranked random forest feature importance scores.
selector1 = NoSelect()
selector2 = EnsembleModelFeatureSelector(model=SklearnModel(model='RandomForestRegressor'), n_features_to_select=20)
selectors = [selector1, selector2]

# Finally, we list which metrics we want to evaluate. If none are given, MAST-ML
# will default to evaulating just the mean absolute error. A complete list of 
# metrics can be obtained from calling Metrics()._metric_zoo() in metrics.py.
metrics = ['r2_score', 'mean_absolute_error', 'root_mean_squared_error', 'rmse_over_stdev']



In [170]:
# As a first run, we want to define and run the case where no data split is 
# performed. This represents a full fit to all of the data. We will fit both our
# random forest and Gaussian process model, but just for the case where all
# features are used.

# splitter = NoSplit()
# splitter.evaluate(X=X,
#                   y=y, 
#                   models=models,
#                   preprocessor=preprocessor,
#                   selectors=[selector1],
#                   metrics=metrics,
#                   savepath=savepath,
# #                   X_extra=X_extra,
#                   verbosity=3)

In [171]:
# Next, we want to do something a bit more informative. We will do a random 
# leave-out cross validation test (5-fold CV). MAST-ML will output data and plots
# for each split as well as some more comprehensive analysis performed over all
# splits. The saved model and preprocessor corresponding to the best split will
# also be put in the split directory, which can be imported for use in future
# predictions 

# splitter = SklearnDataSplitter(splitter='RepeatedKFold', n_repeats=1, n_splits=5)
# splitter.evaluate(X=X,
#                   y=y, 
#                   models=models,
#                   preprocessor=preprocessor,
#                   selectors=[selector2],
#                   metrics=metrics,
#                   plots=['Scatter', 'Histogram'],
#                   savepath=savepath,
# #                   X_extra=X_extra,
#                   verbosity=3)

In [172]:
# Randomly leaving out data isn't the only way to assess model performance. This
# dataset has logical groupings based on which element is the host. So, we can
# do another cross validation test where this time we leave out each host once.
# Let's test just our random forest model with the leave out group CV method.

# splitter = SklearnDataSplitter(splitter='LeaveOneGroupOut')
# splitter.evaluate(X=X,
#                   y=y, 
#                   models=[model1],
#                   preprocessor=preprocessor,
#                   groups=groups,
#                   selectors=[selector2],
#                   metrics=metrics,
#                   plots=['Scatter', 'Histogram'],
#                   savepath=savepath,
#                   X_extra=X_extra,
#                   verbosity=3)

In [173]:
# Now lets use just random forest and the full feature set to do random leave out CV, 
# but let's do a nested variant so we can recalibrate the predicted random forest errors.
# This will allow us to more deeply assess the uncertainty estimates of the 
# random forest model, and see that they would benefit from a correction scheme,
# which in this case is applied following the method of Palmer et al.
#

splitter = SklearnDataSplitter(splitter='RepeatedKFold', n_repeats=1, n_splits=3)
splitter.evaluate(X=X,
                  y=y, 
                  models=[model1],
                  preprocessor=preprocessor,
                  selectors=[selector1],
                  metrics=metrics,
                  plots=['Error'],
                  savepath=savepath,
                  leaveout_inds = X_test, # IMPORTANT, if we include leaveout_inds, we specify test data for MASTAML
                  #X_extra=X_extra,
                  nested_CV= False, # I turn this off to make sure that it does not do nested cross val, reduce time perhaps.
                  error_method='stdev_weak_learners', #stdev_weak_learners, jackknife_after_bootstrap
                  recalibrate_errors=True,
                  verbosity=2)

NLL optimization successful!
NLL optimization successful!


In [55]:
# splitter = SklearnDataSplitter(splitter='RepeatedKFold', n_repeats=2, n_splits=3)
# splitter.evaluate(X=X,
#                   y=y, 
#                   models=[model1],
#                   preprocessor=preprocessor,
#                   selectors=[selector1],
#                   metrics=metrics,
#                   plots=['Error'],
#                   savepath=savepath,
# #                   leaveout_inds = X_test,
#                   #X_extra=X_extra,
#                   nested_CV= True,
#                   error_method='stdev_weak_learners', #stdev_weak_learners, jackknife_after_bootstrap
#                   recalibrate_errors=True,
#                   verbosity=2)

NLL optimization successful!
NLL optimization successful!
NLL optimization successful!
NLL optimization successful!
NLL optimization successful!
NLL optimization successful!
NLL optimization successful!
NLL optimization successful!
NLL optimization successful!
NLL optimization successful!
NLL optimization successful!
NLL optimization successful!


In [52]:
cali_mod_err_path = '/Users/bernie/Desktop/course_material/SP21/SKUNK/week7/save_03_06_18_02_15/RandomForestRegressor_RepeatedKFold_SklearnPreprocessor_NoSelect_03_06_18_02_48/model_errors_leaveout_calibrated.xlsx'
cali_mod_err= pd.read_excel(cali_mod_err_path)
cali_mod_res_path = '/Users/bernie/Desktop/course_material/SP21/SKUNK/week7/save_03_06_18_02_15/RandomForestRegressor_RepeatedKFold_SklearnPreprocessor_NoSelect_03_06_18_02_48/residuals_leaveout.xlsx'
res = pd.read_excel(cali_mod_res_path)

In [53]:
cali_mod_err

,model_errors
0,1.733700
1,1.643457
2,1.682653
3,1.637061
4,1.794367
...,...
995,1.636883
996,1.596815
997,1.657442
998,1.613850


In [54]:
res

,residuals
0,-2.331637
1,0.807427
2,2.347756
3,-0.618770
4,0.176837
...,...
995,-1.559272
996,-4.007684
997,0.353475
998,-0.433831


In [56]:
X

,x0,x1,x2,x3,x4
0,0.417022,0.720324,0.000114,0.302333,0.146756
1,0.092339,0.186260,0.345561,0.396767,0.538817
2,0.419195,0.685220,0.204452,0.878117,0.027388
3,0.670468,0.417305,0.558690,0.140387,0.198101
4,0.800745,0.968262,0.313424,0.692323,0.876389
...,...,...,...,...,...
995,0.489336,0.486921,0.570292,0.544508,0.917111
996,0.884385,0.783719,0.322727,0.784437,0.145185
997,0.860226,0.773000,0.775967,0.528699,0.450185
998,0.226631,0.808678,0.965929,0.543532,0.902207


In [57]:
print('1')

1
